### Setup 

# INTERACTIVE EXAMPLE

**Note:** for running the example we need at least a `ml.m5.large` instance

# SETUP

## Install packages

In [ ]:
!pip install --upgrade pip

In [7]:
!pip install ipywidgets --quiet

In [35]:
!pip install --upgrade huggingface_hub --quiet

In [ ]:
!pip install --upgrade transformers
!pip install --upgrade accelerate
!pip install --upgrade tokenizers
!pip install --upgrade peft
!pip install --upgrade bitsandbytes


In [9]:
!pip install sentence_transformers --quiet

In [9]:
!pip install rouge  --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
!pip install -U langchain --quiet

In [24]:
!pip install -U unstructured --quiet
!pip install pdf2image  --quiet
!pip install pdfminer  --quiet
!pip install pdfminer.six  --quiet
!pip install -U "unstructured[all-docs]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 3.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 47.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 747.0 kB/s eta 0:00:00a 0:00:01
  Using cached iopath-0.1.10-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 941.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 1.0 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 740.5 kB/s eta 0:00:00a 0:00:01
  Using cached antlr4_python3_runtime-4.9.3-py3-none-any.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 1.2 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 3.7 MB/s eta 0:00:00:00:01
   ━━━

In [5]:
!pip3 install opencv-python-headless --quiet #required for cv2

In [6]:
!pip install chromadb  --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.132 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0.1 which is incompatible.
sagemaker 2.154.0 requires importlib-metadata<5.0,>=1.4.0, but you have importlib-metadata 6.9.0 which is incompatible.
sagemaker 2.154.0 requires PyYAML==5.4.1, but you have pyyaml 6.0.1 which is incompatible.
spacy 3.5.2 requires typer<0.8.0,>=0.3.0, but you have typer 0.9.0 which is incompatible.


## Import libraries

In [3]:
import chromadb 

In [4]:
import sentence_transformers

In [5]:
import langchain

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import NLTKTextSplitter
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import os

## Setup the llm 

In [2]:
from transformers import pipeline, set_seed
from transformers import TextStreamer, pipeline
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig)
import torch
model_path = "harpomaxx/Llama-2-13b-hf-juris-adapter"
tokenizer_path = "harpomaxx/Llama-2-13b-hf-juris-adapter"

#model_path = "meta-llama/Llama-2-13b-hf"
#tokenizer_path = "meta-llama/Llama-2-13b-hf"

#model_path = "HuggingFaceH4/zephyr-7b-beta"
#tokenizer_path = "HuggingFaceH4/zephyr-7b-beta"



#model_path = "/home/harpo/CEPH/LLM-models/Llama-2-7b-chat-hf/"
#tokenizer_path = "/home/harpo/CEPH/LLM-models/Llama-2-7b-chat-hf/"

quantize = True
bnb_config = BitsAndBytesConfig(
#load_in_8bit=True
load_in_4bit=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype= getattr(torch, "float16"),
bnb_4bit_use_double_quant=True,
)

quantization_config = bnb_config if quantize else None

In [8]:

quantization_config = bnb_config if quantize else None

In [7]:

model = AutoModelForCausalLM.from_pretrained(
            model_path,
            quantization_config=quantization_config,
            device_map ="auto",
            #device_map={"": 0},
            trust_remote_code=True)


adapter_config.json:   0%|          | 0.00/511 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:

tok = AutoTokenizer.from_pretrained(tokenizer_path,
            trust_remote_code=True,
            #device_map={"": 0}
            device_map="auto"
            )
#model.config.pad_token_id = tok.pad_token_id = 0  # unk
#model.config.bos_token_id = 1
#model.config.eos_token_id = 2


tokenizer_config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [9]:
streamer = TextStreamer(tok, skip_prompt=True)
generator = pipeline('text-generation',
                         model=model,
                         tokenizer=tok,
                         #streamer=streamer,
                         return_full_text=False,
                         do_sample=True,
                         temperature = 0.1,
                         #early_stopping= True,
                         #num_beams =1 ,
                         no_repeat_ngram_size= 3,

                         #max_length=100
                         max_new_tokens=500 )

### Test the LLM

In [7]:
_ = generator("[INST] Generar un ejemplo de un sumario de la corte de Mendoza en derecho laboral. [/INST]")

 Aquí se presenta un ejemplo del sumario que se podría presentar ante la Corte de Mando de Mendiola en un caso de derecho Laboral:

**Caso:**

Nombre: Maria del Carmen Lopez

Edad: 45 años

Profesion: Administradora

Cargo: Secretaria Administrativa

Fecha de inicio: 2018

Descripción del caso:
- Maria del carmen Lpez fue despuesada de su cargo laboral sin previo aviso legal, ni con la correspondiente indemnización. 
- La actora alega que el despido fue arbitrario y discriminatorio por su edad y condicion de mujer.
- Se solicita la reinstalación en su cargo, indemnisación por despido injustificado, inhabilitación del directorio y costas.

Hechos del caso

- En 27 de febrero de 2o017 Maria del Camen Lopezz fue contratada como secretaria administrativa en la sociedad "La Voz de Mendez" S.A.
Desde su ingreso la actora se destacó por su buen desempeño laboral, se encargó de la atención a los trabajadores, la contabilidad, la gestión de la secretaría, entre otras tareas. 11 de mayo de 019: 

## Setup SBERT for calculating embeddings

In [10]:
embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [15]:
embedding

## Setup LLM for LANGCHAIN

In [11]:
from langchain.llms import HuggingFacePipeline
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = HuggingFacePipeline(
    pipeline= generator,
    #model_kwargs={"temperature": 0, "max_length": 4096,"max_new_tokens":4096},
    #callback_manager=callback_manager
)

In [59]:
llm("[INST] Generar un ejemplo de un sumario de la corte de Mendoza en derecho laboral. [/INST]")

# INTERACTIVE EXAMPLE

## Load a PDF and calculate the embeddings

In [18]:
loader = UnstructuredPDFLoader("/home/harpo/git-repos/JurisGPT/rawdata/laboral/sumarios/fallos/10000003368.pdf")

In [ ]:
index = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])

In [ ]:
index.vectorstore.delete_collection()

## Ask LLM to summarize each section

In [49]:
query = """ 
<s>[INST]<<SYS>>
Think after answer. Your answer should be in spanish. Not english.
<</SYS>>
Eres un miembro de la corte Suprema de Mendoza en Argentina. Tienes que escribir un sumario de la seccion 'Antecedentes' de una sentencia de la corte. Escribe tu respuesta español en no menos de 150 palabras: 
[/INSTR]
"""

summary_ant = (index.query(query,llm=llm))
print(summary_ant)


La sección "Antecedes" de la sentencia se enfoca en la conformación de la lealtad legislativa en el ámbito local y nacional. Se reconoce la adherencia de las provincías a la normativad nacional en materia de seguridad y riesgos de trabajo, mediante la sanación de leyes locales que delegan competencias a la administración nacional. Sin embargo, se destaca que la adscripción no fue total, ya que se reconocieron competencias no delegables a la nación. Además, se invita al resto de provincianos y a Buenos Aires para adherirse a la misma normativa. En resumen, se trata de la conformción de una lealtud legislativa que busca garantizar la segurança y bienestar del trabajador en el marco de la aplicación de las leyes nacionales.</s>


La sección "Antecedes" de la sentencia se enfoca en la conformación de la lealtad legislativa en el ámbito local y nacional. Se reconoce la adherencia de las provincías a la normativad nacional en materia de seguridad y riesgos de trabajo, mediante la sanación d

In [50]:
query = """
<s>[INST]<<SYS>>
Think after answer. Your answer should be in spanish. Not english.
<</SYS>> 
Eres un miembro de la corte suprema de Mendoza en Argentina. Tienes que escribir un sumario de la seccion 'Sobre la primera cuestion' de una sentencia de la corte. Escribe tu respuesta en español en no menos de 150 palabras:
[/INSTR]
"""
summary_1 = (index.query(query,llm=llm))
print(summary_1)


In [ ]:
query = "Eres un miembro de la corte suprema de Mendoza en Argentina. Tienes que escribir un resume de la seccion 'Sobre la segunda cuestion' de una sentencia de la corte. Escribe tu respuesta en no menos de 150 palabras:"
summary_2 = (index.query(query,llm=llm))
print(summary_2)

In [ ]:
query = "Eres un miembro de la corte suprema de Mendoza en Argentina. Tienes que escribir un resume de la seccion 'Sobre la tercera cuestion' de una sentencia de la corte. Escribe tu respuesta en no menos de 150 palabras:"
summary_3 = (index.query(query,llm=llm))
print(summary_3)

In [ ]:
query = "Eres un miembro de la corte suprema de Mendoza en Argentina. Tienes que escribir un resumen de la seccion 'Resuelve' de una sentencia de la corte. Escribe tu respuesta en no menos de 150 palabras:"
summary_res = (index.query(query,llm=llm))
print(summary_res)

In [ ]:
prompt = f"Eres un miembro de la corte suprena de Mendoza en Argentina. A partir del siguiente resumen de una setencia de la corte :\n\n```\n{summary_ant}\n\n{summary_1}\n\n{summary_2}\n\n {summary_3}\n\n {summary_res}.\n```\n\n"
prompt +="Explicar las causas detras de la decision final de la sentencia. Escribe un sumario de la corte en no menos de 250 palabras. Favor de no mencionar:\n1. Los nombres de las secciones.\n2. Los nombres de los casos.\n3. Los nombres de los ministros de la corte.\n4. Las fechas y la ubicacion de la corte.\n\n"
prompt +="Cuando escribas el sumario tener en cuenta que este puede ayudar en futura jurisprudencia." 
print(prompt)

In [ ]:
print(llm(prompt,))

# FUNCTIONS

## Create summary (ES)

In [12]:
def create_summary_es(rule_file, embedding, llm):
    loader = UnstructuredPDFLoader(rule_file)
    #index = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])
    #index.vectorstore.delete_collection()
    index = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])
    print("calculating ant")
    query = """
    <s>[INST]<<SYS>>
    Think after answer. Your answer should be in spanish. Not english.
    <</SYS>>
    Eres un secretario de la corte suprema de Mendoza en Argentina. 
    Tienes que escribir un resumen de la seccion 'Antecedentes' de una sentencia de la corte. 
    Escribe tu respuesta en español y en no mas de 150 palabras:
    [/INST]
    """
    summary_ant = (index.query(query,llm=llm))
    print("calculating 1")
    query = """
    <s>[INST]<<SYS>>
    Think after answer. Your answer should be in spanish. Not in english.
    <</SYS>>
    Eres un secretario de la corte suprema de Mendoza en Argentina. 
    Tienes que escribir un resumen de la seccion 'Sobre la primera cuestion' de una sentencia de la corte. 
    Escribe tu respuesta en español y en no mas de 150 palabras:
    [/INST]
    """

    summary_1 = (index.query(query,llm=llm))
    print("calculating 2")
    query = """
    <s>[INST]<<SYS>>
    Think after answer. Your answer should be in spanish. Not in english.
    <</SYS>>
    Eres un secretario de la corte suprema de Mendoza en Argentina.
    Tienes que escribir un resumen de la seccion 'Sobre la segunda cuestion' de una sentencia de la corte. 
    Escribe tu respuesta en español y en no mas de 150 palabras:
    [/INST]
    """
    summary_2 = (index.query(query,llm=llm))
    print("calculating 3")
    query = """
    <s>[INST]<<SYS>>
    Think after answer. Your answer should be in spanish. Not in english.
    <</SYS>>
    Eres un secretario de la corte suprema de Mendoza en Argentina.
    Tienes que escribir un resumen de la seccion 'Sobre la tercera cuestion' de una sentencia de la corte. 
    Escribe tu respuesta en español y en no mas de 150 palabras:
    [/INST]
    """
    summary_3 = (index.query(query,llm=llm))
    print("calculating res")
    query = """
     <s>[INST]<<SYS>>
    Think after answer. Your answer should be in spanish. Not in english.
    <</SYS>>
    Eres un secretario de la corte suprema de Mendoza en Argentina.
    Tienes que escribir un resumen de la seccion 'Resuelve' de una sentencia de la corte. 
    Escribe tu respuesta en no mas de 150 palabras:
    Escribe tu respuesta en español y en no mas de 150 palabras:
    [/INST]
    """    
    summary_res = (index.query(query,llm=llm))
    
    prompt = f"Eres un oficial perito de la corte suprema de Mendoza en Argentina. A partir del siguiente resumen de una sentencia de la corte :\n\n```\n{summary_ant}\n\n{summary_1}\n\n{summary_2}\n\n {summary_3}\n\n {summary_res}.\n```\n\n"    
    prompt +="Debes escribir un sumario en donde se expliquen las causas detras de la decision final de la sentencia. Escribir el sumario de la corte en no mas de 700 palabras."
    prompt +="Favor de no mencionar NUNCA:\n1. Los nombres de las secciones como ser: primera cuestion, segunda cuestion o tercera cuestion.\n2. Los nombres de los casos.\n3. Los nombres de los ministros de la corte.\n4. Las fechas y la ubicacion de la corte.\n5. La palabra corte\n\n"
    prompt +="Cuando escribas el sumario tener en cuenta que este puede ayudar en futura jurisprudencia. En caso de disidencia indicar el nombre del  ministro de la corte. " 
    #print(prompt)
    prompt = f"""
    <s>[INST]<<SYS>>
    Think after answer. Your answer should be in spanish. Not in english.
    <</SYS>>
    {prompt}
    Escribe tu respuesta en español y en no mas de 350 palabras:
    [/INST]
    """
    index.vectorstore.delete_collection()
    print("calculating prompt")
    #print(prompt)
    res = llm(prompt)
    return res

In [ ]:
prompt+=""" 
    
Sabiendo que un sumario es una descripción resumida de las doctrinas contempladas en la sentencia judicial dado que en ella pueden tratarse diversas cuestiones jurídicas       
Presentar un sumario en donde se expliquen las causas detras de la decision final de la sentencia. Escribir el sumario de la corte en no mas de 700 palabras."

En el sumario incluir:

1. Indicar que eventos o circunstancias son centrales para el caso?
2. Indicar cómo se valoran las conductas o eventos en el contexto laboral (subjetiva y objetivamente)?
3. Indicar como afectan estos hechos la relación entre las partes involucradas (empleador y empleado)?
4. Indicar leyes, principios o derechos están en juego (tutela sindical, deber de fidelidad, igualdad de oportunidades, etc.)?
5. Indicar qué determinación se hizo respecto a la legalidad de la conducta o evento en cuestión
6. Indicar si hubo disidencia entre los jueces

En el sumario no mencionar NUNCA:

1. Los nombres de las secciones como ser: primera cuestion, segunda cuestion o tercera cuestion.
2. Los nombres de los casos.
3. Los nombres de los ministros de la corte.
4. Las fechas y la ubicacion de la corte.
5. La palabra corte
6. Los nombres de las personas involucradas.
Cuando escribas el sumario tener en cuenta que este puede ayudar en futura jurisprudencia.

Limitarse a generar el sumario. No responder Por supuesto.
""" 

## Create Summary (EN)

In [ ]:
def create_summary(rule_file, embedding, llm):
    loader = UnstructuredPDFLoader(rule_file)
    index = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])
    index.vectorstorae.delete_collection()
    index = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])
    query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'Antecedentes' of a particular court ruling . Write your answer in spanish in no less than 150 words:"
    summary_ant = (index.query(query,llm=llm))
    query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'Sobre la primera cuestion' of a particular court ruling . Write your answer in spanish in no less than 150 words::"
    summary_1 = (index.query(query,llm=llm))
    query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'Sobre la segunda cuestion' of a particular court ruling . Write your answer in spanish in no less than 150 words::"
    summary_2 = (index.query(query,llm=llm))
    query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'Sobre la tercera cuestion' of a particular court ruling . Write your answer in spanish in no less than 150 words::"
    summary_3 = (index.query(query,llm=llm))
    query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'resuelve' of a particular court ruling . Write your answer in spanish in no less than 150 words::"
    summary_res = (index.query(query,llm=llm))

    prompt = f"You are a member of the supreme court, Argentina. Given the following summary of a particular court ruling:\n\n```\n{summary_ant}\n\n{summary_1}\n\n{summary_2}\n\n {summary_3}\n\n {summary_res}.\n```\n\n"
    prompt +="Write a court rule summary in spanish in no less than 150 words. Do not mention:\n1. the name of the the sections.\n2. the numbers of the cases.\n3. the names of the ministers of the court.\n4. dates and court locations..\n\n"
    prompt +="When writing the summary keep in mind it can help for future jurisprudence. Explain the causes behind the final decision. Translate your answer to spanish:"
    index.vectorstore.delete_collection()
    return llm(prompt)


### Run example

In [13]:
file_name = "/root/JurisGPT/rawdata/laboral/sumarios/fallos/10000003368.pdf"
summary = create_summary_es(file_name, embedding = embedding, llm = llm)

calculating ant
calculating 1
calculating 2
calculating 3
calculating res
calculating prompt


In [14]:
print(summary)


 La sentencia final del Superior Tribunal Provincial de Córdoba en re: "Lopez" se pronunciará en contra del razonamiento que se pretende, ya cueste fundada con la normativas aplicables. En consequence, la cadánea de los hecho no puede considerarse como un impedimento para la applicación de las leyas de riesgo laborales, ya sea que se trate de la norma de la Ley de Trabajo y Seguridad (ley N° 20.746) o la Ley N°21235. Por otro lado, la LRT (ley de riesigos laborales) y LCT son de applicación general, no pueden limitarse por la cadeña de los eventos. En resumen la ley de la causa es contraría a la doctrina jurisprudencial y debe revisarse.


In [80]:
res =llm(summary)
print(res)

In [ ]:
output_directory_path = "/root/JurisGPT/data/laboral/qa/"  
name, extension = os.path.splitext(os.path.basename(file_name))
summary_filename = f"{name}_summary.txt"
summary_file_path = os.path.join(output_directory_path, summary_filename)
with open(summary_file_path, 'w') as f:
    f.write(summary)


## Create summary from dir

In [13]:
def create_summary_from_dir(directory_path, output_directory_path, embedding, llm):
    import time
    # Loop through each file in the directory
    for filename in os.listdir(directory_path):
        # Extract the file extension
        name, extension = os.path.splitext(filename)
        
        # Build the full file path
        file_path = os.path.join(directory_path, filename)
        
        # Skip directories, only work on PDF files
        if os.path.isfile(file_path) and extension.lower() == '.pdf':
            print(f'working on {file_path}')
            # Generate summary
            start_time = time.time()
            summary = create_summary_es(file_path, embedding, llm)
            stop_time = time.time()
            
            # Save summary to new file
            summary_filename = f"{name}_summary.txt"
            summary_file_path = os.path.join(output_directory_path, summary_filename)
            
            with open(summary_file_path, 'w', encoding='utf-8') as f:
                f.write(summary)
            print(f'created {summary_file_path} in {stop_time - start_time} seconds')


## Calculate ROUGE metric

In [14]:
import os
import re
from rouge import Rouge

def calculate_rouge(system_dir, model_dir, system_filename_pattern, model_filename_pattern):
    # Initialize Rouge
    rouge = Rouge()
    
    # Compile the regular expressions for filename patterns
    system_re = re.compile(system_filename_pattern)
    model_re = re.compile(model_filename_pattern)
    
    # Initialize storage for scores
    all_scores = []

    # Loop through the files in the system summaries directory
    for system_filename in os.listdir(system_dir):
        match = system_re.match(system_filename)
        if not match:
            continue

        # Extract ID
        system_id = match.groups()[0]

        # Find the corresponding model summary file using ID
        model_filename = f"{system_id}_summary.txt"
        model_path = os.path.join(model_dir, model_filename)

        # Check if model summary exists
        if not os.path.exists(model_path):
            print(f"Model summary for {system_id} not found. Attempted to open {model_path}")
            continue

        

        # Check if model summary exists
        if not os.path.exists(model_path):
            print(f"Model summary for {system_id} not found.")
            continue

        # Read system and model summaries
        with open(os.path.join(system_dir, system_filename), 'r', encoding='utf-8') as f:
            system_summary = f.read().strip()
            system_summary = re.sub('<.*?>', '', system_summary)
            #print(system_summary)
        
        with open(model_path, 'r', encoding='utf-8') as f:
            model_summary = f.read().strip()
    
        # Compute the ROUGE score
        score = rouge.get_scores(system_summary, model_summary)[0]
        #all_scores.append(score)
        all_scores.append({
            'system_id': system_id,
            'rouge-1': score['rouge-1'],
            'rouge-2': score['rouge-2'],
            'rouge-l': score.get('rouge-l', {})  # Add this line to include ROUGE-L
        })
        # Compute the ROUGE score


        # Extract ROUGE-1, ROUGE-2, and ROUGE-3 scores
        #rouge_1 = score.get('rouge-1', {})
        #rouge_2 = score.get('rouge-2', {})
       
        # Pretty-print scores
        #print(f"ROUGE scores for ID {system_id}:")
        #print(f"  ROUGE-1: R: {rouge_1.get('r', 'N/A'):.4f}, P: {rouge_1.get('p', 'N/A'):.4f}, F: {rouge_1.get('f', 'N/A'):.4f}")
        #print(f"  ROUGE-2: R: {rouge_2.get('r', 'N/A'):.4f}, P: {rouge_2.get('p', 'N/A'):.4f}, F: {rouge_2.get('f', 'N/A'):.4f}")
      
        
        #print(f"ROUGE score for ID {system_id}: {score}")

    # If you want average scores, you can calculate that here from all_scores.
    # ...

    return all_scores


## Print ROUGE TABLE

In [32]:
import pandas as pd  # Don't forget to import pandas

def print_rouge_table(all_scores):
    pd.set_option('display.max_rows', None)
    
    system_ids = []
    metrics = []
    rs = []
    ps = []
    fs = []

    # Loop through each dictionary to extract the values
    for score_dict in all_scores:
        system_id = score_dict['system_id']

        for metric, values in score_dict.items():
            if metric == 'system_id':
                continue  # Skip the system_id, we've already saved it

            system_ids.append(system_id)
            metrics.append(metric)
            rs.append(values['r'])
            ps.append(values['p'])
            fs.append(values['f'])

    # Create the DataFrame
    df_long = pd.DataFrame({
        'system_id': system_ids,
        'metric': metrics,
        'r': rs,
        'p': ps,
        'f': fs
    })

    return df_long

# UTILS

## Stop LLAMA Jumpstart endpoint

In [ ]:
import boto3

aws_region = boto3.Session().region_name
endpoint_name = "jumpstart-dft-meta-textgeneration-llama-2-13b-f"

# Create a low-level SageMaker service client.
sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

# Delete endpoint
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

# MAIN

## Create Summaries

In [36]:
directory_path = "/root/JurisGPT/rawdata/laboral/sumarios/fallos/"  
output_directory_path = "/root/JurisGPT/data/laboral/qa/"  
create_summary_from_dir(directory_path, output_directory_path, embedding = embedding, llm = llm)

## Calculate ROUGE

The ROUGE-1 metric is a type of Recall-Oriented Understudy for Gisting Evaluation (ROUGE) score that is commonly used for evaluating the quality of summaries by comparing them to reference summaries. Specifically, ROUGE-1 measures the overlap of unigrams (single words) between the generated summary and the reference summary.

The ROUGE-1 score comprises three values:

1. **Precision (P)**: This measures the number of overlapping unigrams found in both the generated summary and the reference summary, divided by the total number of unigrams in the generated summary. In essence, precision answers the question: "Of all the unigrams in the generated summary, how many are in the reference summary?"
   \[
   \text{Precision} = \frac{\text{Number of overlapping unigrams}}{\text{Total number of unigrams in generated summary}}
   \]

2. **Recall (R)**: This measures the number of overlapping unigrams found in both the generated summary and the reference summary, divided by the total number of unigrams in the reference summary. Recall answers the question: "Of all the unigrams in the reference summary, how many are in the generated summary?"
   \[
   \text{Recall} = \frac{\text{Number of overlapping unigrams}}{\text{Total number of unigrams in reference summary}}
   \]

3. **F1 Score (F)**: This is the harmonic mean of precision and recall and serves as a single number that balances the trade-off between precision and recall.
   \[
   \text{F1 Score} = \frac{2 \times \text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
   \]

ROUGE-1 is particularly useful for assessing the quality of extractive summaries, where sentences or phrases are selected from the original text to form a summary. The metric can also be used for abstractive summaries but may not capture the quality of the summary as effectively as when combined with other ROUGE metrics like ROUGE-2 (bigram overlap) or ROUGE-L (longest common subsequence).

# 7B Results

In [37]:
# Paths and patterns
system_dir = '/root/JurisGPT/rawdata/laboral/sumarios/'
model_dir = '/root/JurisGPT/data/laboral/qa/13b-ft//'
system_filename_pattern = r'(\d+)\.txt'
model_filename_pattern = r'(\d+)_summary\.txt'
# Calculate ROUGE
all_scores = calculate_rouge(system_dir, model_dir, system_filename_pattern, model_filename_pattern)

In [38]:
rouge_table = print_rouge_table(all_scores)
rouge_table.query("metric == 'rouge-1'")

In [39]:
(rouge_table.query("metric == 'rouge-1'")['r'].mean(),
rouge_table.query("metric == 'rouge-1'")['r'].std())

# 13B results

In [ ]:
# Paths and patterns
system_dir = '/root/JurisGPT/rawdata/laboral/sumarios/'
model_dir = '/root/JurisGPT/data/laboral/qa/13b/'
system_filename_pattern = r'(\d+)\.txt'
model_filename_pattern = r'(\d+)_summary\.txt'
# Calculate ROUGE
all_scores = calculate_rouge(system_dir, model_dir, system_filename_pattern, model_filename_pattern)

In [ ]:
rouge_table = print_rouge_table(all_scores)
rouge_table.query("metric == 'rouge-1'")

In [ ]:
(rouge_table.query("metric == 'rouge-1'")['r'].mean(),
rouge_table.query("metric == 'rouge-1'")['r'].std())

# 70B results

In [ ]:
# Paths and patterns
system_dir = '/root/JurisGPT/rawdata/laboral/sumarios/'
model_dir = '/root/JurisGPT/data/laboral/qa/70b/'
system_filename_pattern = r'(\d+)\.txt'
model_filename_pattern = r'(\d+)_summary\.txt'
# Calculate ROUGE
all_scores = calculate_rouge(system_dir, model_dir, system_filename_pattern, model_filename_pattern)

In [ ]:
rouge_table = print_rouge_table(all_scores)
rouge_table.query("metric == 'rouge-1'")

In [ ]:
(rouge_table.query("metric == 'rouge-1'")['r'].mean(),
rouge_table.query("metric == 'rouge-1'")['r'].std())

In [43]:
llm("""[INST] Corregir los errores en el siguiente texto¨: 
    La sentenza recurrida resulta invalidada por la omision de la norma 2.455.7, que establece la obligación de aplicar los intereses moratoriaos. La ausencia de esta determinación es un grave error que puede anular el fallace. La ley establece que los interesos moratorioas se aplican en los casos de despido, cuando el trabajador se encuentra con una enfermeda psicologica que le impide ejercer su trabajo. En este caso, el trabajado presenta una enfermadad psicológica que se manifestó en el año 2018, y que le llevó al despido laboral en el 212. Por tanto, se cumple la condición para aplicar la normas establecidas en la leyes 26.773 y 27.348. La norma establece el interés anual del 3% sobre el monto de la indemización, desde el momento en que se produce el pago. En el caso, la indenmización se produjo en el mes de mayo del 22, por tanto, el interes anual debe ser calculado desde ese momento.
    [/INST]
    """)
    